In [ ]:
!pwd

In [ ]:
%conda install python-dotenv wandb nbstripout -y

# Setting lerobot env
from lerobot_util import setup_lerobot_env

setup_lerobot_env()

In [ ]:
# Run the following command to identify USB port for arms
from lerobot_util import print_shell_md
import sys

!v4l2-ctl --list-devices

print_shell_md(
  "Run following command to identify USB port for arms.", 
  f"{sys.executable}",
  "src/lerobot/scripts/lerobot_find_port.py"
)

print_shell_md(
  "Run following command to identify camera index.", 
  f"{sys.executable}",
  "src/lerobot/scripts/lerobot_find_cameras.py"
)


In [ ]:
import sys
import json

from lerobot_util import print_shell_md, get_lerobot_camera_index

leader_port = '/dev/ttyACM1'
follower_port = '/dev/ttyACM0'

hf_namespace = 'fanaustinca'
data_repo = f"{hf_namespace}/fll_training_data"

top_camera_index, wrist_camera_index = get_lerobot_camera_index()

# Zoom in 130 to focus on board
!v4l2-ctl -d /dev/video{top_camera_index} --set-ctrl=zoom_absolute=130

camera_config = {
  "wrist": {
    "type": "opencv", 
    "index_or_path": wrist_camera_index, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  },
  "top": {
    "type": "opencv", 
    "index_or_path": top_camera_index, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  }
}

camera_config_str = json.dumps(camera_config, separators=(',', ':'))

arguments = [
    "src/lerobot/scripts/lerobot_teleoperate.py",
    "--robot.type=so101_follower",
    f"--robot.port={follower_port}",
    "--teleop.type=so101_leader",
    f"--teleop.port={leader_port}",
    f"--robot.cameras=\'{camera_config_str}\'",
    "--display_data=true"
]

print_shell_md(
  "Run following command to operate leader arm to control follower arm.", 
  f"{sys.executable}",
  *arguments
)

arguments = [
  "src/lerobot/scripts/lerobot_record.py",
  "--robot.type=so101_follower",
  f"--robot.port={follower_port}"
  f"--robot.cameras=\'{camera_config_str}\'",
  "--teleop.type=so101_leader",
  f"--teleop.port={leader_port}"
  f"--dataset.repo_id={data_repo}",
  "--dataset.num_episodes=50",
  "--dataset.single_task=fll_collection",
  "--dataset.push_to_hub=true",
  "--dataset.private=false"
]

print_shell_md(
  "Run following command to collect training data.", 
  f"{sys.executable}",
  *arguments
)
